# CHATBOT

In [123]:
import os
from dotenv import load_dotenv
import requests

from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display

#import the LLM APIs
import ollama
from openai import OpenAI
import anthropic
import google.generativeai


import gradio as gr  

load_dotenv()
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY=os.getenv("ANTHROPIC_API_KEY")
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")


In [38]:
openAI = OpenAI()
clade= anthropic.Anthropic()
#google = generativeai.configure()

# Classes and Functions

In [6]:
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


class Website:
    def __init__ (self,url):
        ''' 
        Create this website object from the given url using BwautifulSoup
        '''

        self.url=url
        response = requests.get(url,headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No Title found"
        for irrelevant in soup.body(['style', "scripts", "img","input"]):
            irrelevant.decompose()
        self.text =soup.body.get_text(separator="\n",strip="True")

In [124]:
def GenerateLLAMAResponse_stream(text):
    OLLAMA_API = "http://localhost:11434/api/chat"
    HEADERS = {"Content-Type": "application/json"}
    MODEL = "llama3.2"
    messages = [{"role":"system", "content":"You are an assistant that anal yzes the contents of a website \
                    and provides a short summary, ignoring text that might be navigation related. \
                    Respond in markdown."},
                {"role":"user","content":f'''Analyse the given text in triple ticks carefully and give a crips summary ```{text}```'''}]

    payload = {
        "model": MODEL,
        "messages": messages 
    }

    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
    
    return response.json()['message']['content']

## Functions for LLAMA(Local)

In [167]:
def GenerateLLAMAResponse(text):
    OLLAMA_API = "http://localhost:11434/api/chat"
    HEADERS = {"Content-Type": "application/json"}
    MODEL = "llama3.2"
    messages = [{"role":"system", "content":"You are an assistant that anal yzes the contents of a website \
                    and provides a short summary, ignoring text that might be navigation related. \
                    Respond in markdown."},
                {"role":"user","content":f'''Analyse the given text in triple ticks carefully and give a crips summary ```{text}```'''}]

    payload = {
        "model": MODEL,
        "messages": messages
    }

    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
    
    return response.json()['message']['content']
   

## Functions for Google

In [261]:
def Get_Google_Response(messages):
   
    gemini_via_openai_client = OpenAI(
        api_key=GOOGLE_API_KEY, 
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )
    
    response = gemini_via_openai_client.chat.completions.create(
        model="gemini-1.5-flash",
        messages=messages 
         
    )
    return response

In [171]:
def GenerateGoogleResponse(messages):
   
    gemini_via_openai_client = OpenAI(
        api_key=GOOGLE_API_KEY, 
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )
    
    response = gemini_via_openai_client.chat.completions.create(
        model="gemini-1.5-flash",
        messages=messages 
         
    )
    return response.choices[0].message.content 
    #print(response) 
    #yield response.choices[0].delta.content
    #return response.choices[0].message.content 
    #if stream==False:
     #   return response.choices[0].message.content 
    #else:
     #   yield response.choices[0].delta.content
     

In [166]:
text ="To Be or not to be. That my dear is the question."

messages = [{"role":"system", "content":"You are an assistant that explains the phrase to a 5 year old kid."},
                {"role":"user","content":f'''Analyse the given text in triple ticks carefully and give the explanation ```{text}```'''}]

response= GenerateGoogleResponse(messages,False)
print (response)

Imagine you're playing with your toys, and you have to decide: should you build a big castle, or should you have a spaceship adventure?  That's a question, right?

This sentence, "To be or not to be. That, my dear, is the question," is like that, but it's a much bigger question!  "To be" means to keep living, and "not to be" means to stop living, to die.

So it's asking: "Should I keep living, or should I die? That's the big question!"  It's a question someone might ask when they're feeling very sad or unsure about life.



## Functions for OpenAI

In [99]:
def GenerateOpenAIResponse_stream(messages):
    openAI = OpenAI()
    stream= openAI.chat.completions.create(
            model="gpt-4o-mini", 
            messages=messages,
            stream=True)
    result =""
    print("Received response from GPT")
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        #print(result)
        yield result

In [233]:
def Get_OpenAI_Response(messages,tools,stream=False):
    openAI = OpenAI()
    response= openAI.chat.completions.create(
            model="gpt-4o-mini", 
            messages=messages,
            tools=tools,
            stream=stream)
     
    return response

In [169]:
def GenerateOpenAIResponse(messages,stream):
    openAI = OpenAI()
    response= openAI.chat.completions.create(
            model="gpt-4o-mini", 
            messages=messages,
            stream=stream)
     
    print("Received response from GPT")
     
    if stream==False:
        return response.choices[0].message.content 
    else:
        yield response.choices[0].delta.content   

## Functions to summarise

In [178]:
def call_LLM_Model(model, messages):
    if model=="GPT":
        print("Calling GPT")
        response = GenerateOpenAIResponse(messages)
        print(response)
    elif model =="LLAMA":
        response = GenerateLLAMAResponse(messages)
    elif model =="GEMINI":
        response = GenerateGoogleResponse(messages)

    return response

In [172]:
def summarise (model,text):
    #webText =text #Website(url)
    
    messages = [{"role":"system", "content":"You are an assistant that anal yzes the contents of a website \
                    and provides a short summary, ignoring text that might be navigation related. \
                    Respond in markdown."},
                {"role":"user",
                 "content":f'''Analyse the given text in triple ticks carefully and 
                 give a crips summary ```{text}```'''}]

    
    return call_LLM_Model(model, messages)
        

## Functions to summarise websites

In [179]:
def summarise_webSite (model,url):
    webText =Website(url)
    
    messages = [{"role":"system", "content":"You are an assistant that analyses the contents of a website \
                    and provides a short summary, ignoring text that might be navigation related. \
                    Respond in markdown."},
                {"role":"user",
                 "content":f'''Analyse the given text in triple ticks carefully and 
                 give a crips summary ```{webText.text}```'''}]

    return call_LLM_Model(model, messages)
        

# Generate Summary

In [134]:
#To set dark mode
force_dark_mode="""
function refresh(){
    const url = new URL(window.location)
    if (url.searchParams.get('__theme')!=='dark'){
        url.searchParams.set('__theme','dark')
        window.location.href = url.href
    }
}
"""

## Select Model and summarise text (without Streaming)

In [175]:
#with inputs and outputs

view =gr.Interface (
    fn=summarise,
    inputs =[
        gr.Dropdown(['GPT','LLAMA','GEMINI','CLAUDE'], label="Select the model", value="GEMINI"),
        gr.Textbox(label="Text to summarise", lines =10)      
    ],
    outputs =[gr.Markdown(label="Response:")],
    flagging_mode ="never",
    js=force_dark_mode
)

view.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7897

To create a public link, set `share=True` in `launch()`.


## Summarise WebSites (with Streaming)

In [180]:
#with inputs and outputs

view =gr.Interface (
    fn=summarise_webSite,
    inputs =[
        gr.Dropdown(['GPT','LLAMA','GEMINI','CLAUDE'], label="Select the model", value="LLAMA"),
        gr.Textbox(label="Company URL", lines =1)      
    ],
    outputs =[gr.Markdown(label="Response:")],
    flagging_mode ="never",
    js=force_dark_mode
)

view.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7899

To create a public link, set `share=True` in `launch()`.


## AI Agent for Aurlines FLight Booking (CHATBOT)

In [192]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [194]:
def  chat(message,history):
    messages=[{"role":"system", "content":system_message}] + history +  [{"role":"user", "content":message}]
 
    return GenerateGoogleResponse(messages)

In [195]:
#with inputs and outputs

gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7903

To create a public link, set `share=True` in `launch()`.


### Using Tools

In [259]:
import json
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [251]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [252]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [253]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

In [263]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [264]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": message.tool_calls[0].id
    }
    return response, city

In [265]:
gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7917

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for London
